In [1]:
# ADVANCED MANUFACTURING FRONTIER SECTORS

import pandas as pd
import geopandas as gpd
import numpy as np
import altair as alt
import os
import eco_style 
alt.themes.enable("light")
import requests
import json
import time
from tqdm import tqdm
import io
from shapely.geometry import Point

In [2]:
# IMPORT EMPLOYMENT FOR 5DIG SIC CODES FROM NOMIS API

# FROM NOMIS API RETRIEVE TOTAL EMPLOYMENT FROM BRES FOR LADS

base_url = "https://www.nomisweb.co.uk/api/v01/dataset/NM_189_1.data.csv"

INDUSTRY_LIST_STR = "134218728,134218838,134218848,134218858,134218868,134218878,134218888,134218918,134218938,134218948,134218958,134218968,134218978,134218988,134218998,134219008,134219018,134219028,134219138,134219148,134219158,134219168,134219178,134219188,134219198,134219218,134219228,134219338,134219349,134219357,134219358,134219368,134219428,134219828,134219928,134220028,134220128,134220838,134220848,134220938,134220948,134222829,134222830,134222928,134223828,134223928,134224828,134224938,134225018,134225838,134225848,134226638,134226648,134226658,134226718,134226828,134227628,134227838,134227848,134227858,134227928,134228038,134228048,134228118,134228138,134228148,134228239,134228240,134228247,134228248,134228339,134228340,134228348,134228438,134228448,134228458,134228538,134228549,134228550,134228559,134228560,134228568,134228578,134228588,134228618,134228638,134228648,134228738,134228748,134228758,134228768,134228778,134228788,134228798,134229728,134230828,134230928,134231028,134231638,134231649...134231651,134231659,134231667,134231668,134231678,134231688,134231718,134231838,134231848,134231859,134231860,134231869,134231870,134231918,134231928,134232038,134232118,134232838,134232848,134232928,134233828,134233938,134233948,134233958,134233968,134234018,134234838,134234848,134234939,134234947,134234948,134234958,134234968,134235018,134235838,134235849,134235857,134235858,134235868,134235929...134235931,134236828,134236929,134236937,134237838,134237848,134237858,134237868,134237878,134237888,134237898,134237928,134238029,134238030,134238139,134238140,134238148,134238238,134238248,134238258,134238318,134238328,134238828,134238928,134239838,134239918,134239938,134239948,134239958,134240018,134240838,134240848,134240858,134240868,134240918,134240928,134241038,134241048,134241138,134241148,134241158,134241168,134241218,134241238,134241248,134241338,134241348,134241358,134241368,134241378,134241418,134241428,134241638,134241718,134241828,134241928,134242038,134242048,134242058,134242068,134242138,134242148,134242158,134242168,134242178,134242188,134242238,134242248,134242258,134242268,134242838,134242848,134242938,134243018,134243028,134243128,134243228,134243338,134243348,134243438,134243448,134243458,134243638,134243648,134243658,134243668,134243718,134243838,134243848,134243928,134244029,134244037,134244128,134244239...134244242,134244248,134244328,134244429,134244430,134244528,134244838,134244848,134244928,134245038,134245048,134245058,134245128,134245238,134245248,134245628,134245838,134245848,134245859,134245860,134245868,134245878,134245938,134245948,134245958,134245968,134245978,134246018,134246029,134246030,134246138,134246218,134246638,134246649...134246651,134246658,134246668,134246678,134246688,134246718,134246828,134246929...134246931,134247038,134247048,134247838,134247848,134247928,134248028,134248128,134248638,134248648,134248718,134248738,134248748,134248758,134248818,134249838,134249848,134249858,134249928,134250028,134250129,134250137,134250228,134250638,134250718,134250838,134250848,134250858,134250868,134250878,134250888,134250898,134250918,134250928,134252838,134252848,134252858,134252868,134252938,134252948,134252958,134253028,134253728,134254728,134255838,134255848,134255938,134255948,134256038,134256048,134256728,134258828,134258929,134258930,134259838,134259848,134259858,134259938,134259948,134260638,134260718,134260838,134260848,134260858,134260938,134260948,134261018,134261038,134261048,134261058,134261069,134261070,134261118,134261638,134261719,134261727,134262839,134262840,134262918,134262928,134263038,134263048,134263128,134263838,134263848,134263858,134263868,134263878,134263888,134263898,134263908,134263918,134263938,134263948,134263958,134263968,134264038,134264048,134264058,134264069,134264070,134264078,134264088,134264098,134264108,134264118,134264138,134264148,134264159,134264167,134264168,134264178,134264188,134264198,134264208,134264219,134264227,134264238,134264248,134264338,134264348,134264358,134264368,134264378,134264388,134264418,134264439,134264447,134264448,134264458,134264468,134264478,134264488,134264498,134264628,134264838,134264918,134264938,134264948,134264958,134264968,134264978,134264988,134265018,134265028,134265138,134265149,134265157,134265158,134265238,134265248,134265258,134265268,134265319,134265327,134265338,134265348,134265358,134265368,134265378,134265438,134265449,134265450,134265458,134265469,134265477,134265478,134265488,134265498,134265509,134265510,134265517,134265519,134265527,134265538,134265548,134265618,134265638,134265718,134266828,134266928,134267039,134267047,134267048,134267118,134267138,134267148,134267228,134267828,134267928,134268028,134268128,134268829,134268830,134268938,134268948,134269829...134269831,134269939...134269941,134269947,134269948,134269958,134269969...134269971,134270018,134270828,134270929,134270930,134272828,134272929,134272930,134272937,134273028,134273628,134273829...134273831,134273938,134274018,134274029,134274030,134275838,134275848,134275858,134275869,134275870,134275918,134275938,134276018,134276839...134276841,134276848,134276859...134276861,134276868,134276928,134277828,134277928,134278828,134278928,134279028,134279628,134279739,134279740,134279748,134279758,134279818,134280838,134280848,134281638,134281718,134281838,134281919,134281920,134281929...134281933,134281937,134282029...134282034,134282638,134282649,134282650,134282657,134282719,134282720,134282727,134282838,134282848,134282929,134282930,134283028,134283838,134283848,134283918,134283938,134283948,134284018,134284028,134285828,134285929,134285930,134285937,134286038,134286048,134286829,134286830,134286837,134286929...134286931,134287828,134287938,134287949,134287957,134288839,134288840,134288849,134288850,134288857,134288928,134289838,134289918,134289928,134290838,134290848,134290928,134291828,134291929...134291931,134291937,134292028,134292629,134292630,134292637,134292728,134294838,134294848,134294938,134294948,134295019,134295027,134295038,134295048,134295058,134295069,134295070,134295079,134295080,134295118,134295128,134295829,134295837,134295928,134296028,134296838,134296848,134297629,134297637,134297828,134297928,134298028,134298828,134298938,134298949...134298951,134298957,134299019,134299027,134299028,134299838,134299918,134299928,134300029,134300030,134300639,134300640,134300648,134300718,134301838,134301848,134301858,134301938,134301948,134301958,134301968,134301978,134302028,134302828,134302928,134303038,134303048,134303138,134303149,134303150,134303238,134303248,134303258,134303318,134303328,134303829,134303830,134303938,134303948,134303958,134304628,134304828,134304928,134305028,134305628,134305828,134306638,134306718,134307738,134307748,134307758,134307768,134308739,134308740,134308748,134308758,134308768,134309728,134310838,134310848,134310858,134310919,134310927,134310938,134311018,134311838,134311848,134311928,134312638,134312648,134312718,134312838,134312848,134312938,134312948,134312958,134312968,134312978,134313018,134313738,134313748,134313758,134313768,134313818,134314728,134315828,134315928,134316728"

params = {
        'geography': '2092957698',
        'industry':INDUSTRY_LIST_STR,
        'employment_status': '4',
        'measure':'1',
        'measures': '20100'
    }

response = requests.get(base_url, params=params)

        # Raise an error if the API returns 4xx or 5xx status code
response.raise_for_status()

        # Use io.StringIO to read the text content into a dataframe
chunk_df = pd.read_csv(io.StringIO(response.text))

        # Save the chunk to a file
chunk_df.to_csv('annual_BRES_employment_5DIGSIC.csv', index=False)


In [3]:
# Read in saved df from NOMIS API

employment_SIC = pd.read_csv("annual_BRES_employment_5DIGSIC.csv")

# Select columns of interest
employment_SIC = employment_SIC[["DATE","INDUSTRY_CODE", "OBS_VALUE"]]
employment_SIC = employment_SIC.rename(columns={"OBS_VALUE":"EMPLOYMENT"})

In [4]:
# Add flag for the IS8 sectors

IS8_LOOKUP = pd.read_csv("IS-8 SIC Lookup.csv")

IS8_LOOKUP.head()

# Full join first with employment data to introduce indicator of IS8 sector, later on we filter
# We need to do this on SIC digit level because some are defined at 2,3,4 etc

# Create different digit columns in the employment data before merging
employment_SIC["SIC_2digit"] = employment_SIC["INDUSTRY_CODE"].astype(str).str[:2]
employment_SIC["SIC_3digit"] = employment_SIC["INDUSTRY_CODE"].astype(str).str[:3]
employment_SIC["SIC_4digit"] = employment_SIC["INDUSTRY_CODE"].astype(str).str[:4]
employment_SIC["SIC_5digit"] = employment_SIC["INDUSTRY_CODE"].astype(str).str[:5]

IS8_LOOKUP['SIC'] = IS8_LOOKUP['SIC'].astype(str)
IS8_LOOKUP = IS8_LOOKUP[['SIC','Digit level','IS8 sector']]
IS8_LOOKUP = IS8_LOOKUP.rename(columns={'IS8 sector':'IS8_Sector'})

lookup_2dig = IS8_LOOKUP[IS8_LOOKUP['Digit level'] == 2].drop('Digit level',axis=1)
lookup_3dig = IS8_LOOKUP[IS8_LOOKUP['Digit level'] == 3].drop('Digit level',axis=1)
lookup_4dig = IS8_LOOKUP[IS8_LOOKUP['Digit level'] == 4].drop('Digit level',axis=1)
lookup_5dig = IS8_LOOKUP[IS8_LOOKUP['Digit level'] == 5].drop('Digit level',axis=1)


# 1. Merge 5-digit SICs

df_merged = pd.merge(employment_SIC, lookup_5dig, how='left', left_on='SIC_5digit', right_on='SIC', suffixes=('','_5'))

# 2. Merge 4-digit SICs

df_merged = pd.merge(df_merged, lookup_4dig, how='left', left_on='SIC_4digit', right_on='SIC', suffixes=('','_4'))

# 3. Merge 3-digit SICs

df_merged = pd.merge(df_merged, lookup_3dig, how='left', left_on='SIC_3digit', right_on='SIC', suffixes = ('','_3'))

# 4. Merge 2-digit SICs

df_merged = pd.merge(df_merged, lookup_2dig, how='left', left_on='SIC_2digit', right_on='SIC', suffixes=('','_2'))


# Combine all the digit level flags into one
# Rename the first flag column (from the 5-digit merge) for clarity
df_merged = df_merged.rename(columns={'IS8_Sector': 'IS8_Sector_5'})

# Coalesce the flags, starting with the most specific
df_merged['IS8_Final_Flag'] = (
    df_merged['IS8_Sector_5']
    .fillna(df_merged['IS8_Sector_4'])
    .fillna(df_merged['IS8_Sector_3'])
    .fillna(df_merged['IS8_Sector_2'])
)

# Drop all the intermediary flags
cols_to_drop = [
    'SIC', 'IS8_Sector_5', 
    'SIC_4', 'IS8_Sector_4', 
    'SIC_3', 'IS8_Sector_3', 
    'SIC_2', 'IS8_Sector_2',
]

df_final = df_merged.drop(columns=cols_to_drop, errors='ignore')

df_final.head()

,DATE,INDUSTRY_CODE,EMPLOYMENT,SIC_2digit,SIC_3digit,SIC_4digit,SIC_5digit,IS8_Final_Flag
0,2015,1000,425000,10,100,1000,1000,NaN
1,2015,1110,0,11,111,1110,1110,NaN
2,2015,1120,0,11,112,1120,1120,NaN
3,2015,1130,0,11,113,1130,1130,NaN
4,2015,1140,0,11,114,1140,1140,NaN


In [5]:
# Add frontier sector flag within broader sectors
IS8_LOOKUP = pd.read_csv("IS-8 SIC Lookup.csv")
FRONTIER_LOOKUP = IS8_LOOKUP[IS8_LOOKUP["Frontier sector"].notnull()]

FRONTIER_LOOKUP.head()

# Split SIC into digit level to match
FRONTIER_LOOKUP['SIC'] = FRONTIER_LOOKUP['SIC'].astype(str)
FRONTIER_LOOKUP = FRONTIER_LOOKUP[['SIC','Digit level','Frontier sector']]
FRONTIER_LOOKUP = FRONTIER_LOOKUP.rename(columns={'Frontier sector':'Frontier_Sector'})

lookup_2dig = FRONTIER_LOOKUP[FRONTIER_LOOKUP['Digit level'] == 2].drop('Digit level',axis=1)
lookup_3dig = FRONTIER_LOOKUP[FRONTIER_LOOKUP['Digit level'] == 3].drop('Digit level',axis=1)
lookup_4dig = FRONTIER_LOOKUP[FRONTIER_LOOKUP['Digit level'] == 4].drop('Digit level',axis=1)
lookup_5dig = FRONTIER_LOOKUP[FRONTIER_LOOKUP['Digit level'] == 5].drop('Digit level',axis=1)

# Join to employment df

# 1. Merge 5-digit SICs

df_merged = pd.merge(df_final, lookup_5dig, how='left', left_on='SIC_5digit', right_on='SIC', suffixes=('','_5'))

# 2. Merge 4-digit SICs

df_merged = pd.merge(df_merged, lookup_4dig, how='left', left_on='SIC_4digit', right_on='SIC', suffixes=('','_4'))

# 3. Merge 3-digit SICs

df_merged = pd.merge(df_merged, lookup_3dig, how='left', left_on='SIC_3digit', right_on='SIC', suffixes = ('','_3'))

# 4. Merge 2-digit SICs

df_merged = pd.merge(df_merged, lookup_2dig, how='left', left_on='SIC_2digit', right_on='SIC', suffixes=('','_2'))


# Combine all the digit level flags into one
# Rename the first flag column (from the 5-digit merge) for clarity
df_merged = df_merged.rename(columns={'Frontier_Sector': 'Frontier_Sector_5'})

# Coalesce the flags, starting with the most specific
df_merged['Frontier_Final_Flag'] = (
    df_merged['Frontier_Sector_5']
    .fillna(df_merged['Frontier_Sector_4'])
    .fillna(df_merged['Frontier_Sector_3'])
    .fillna(df_merged['Frontier_Sector_2'])
)

# Drop all the intermediary flags
cols_to_drop = [
    'SIC', 'Frontier_Sector_5', 
    'SIC_4', 'Frontier_Sector_4', 
    'SIC_3', 'Frontier_Sector_3', 
    'SIC_2', 'Frontier_Sector_2',
]

df_final = df_merged.drop(columns=cols_to_drop, errors='ignore')

df_final[df_final['IS8_Final_Flag'].notnull()]

/var/folders/dp/dmj0l1fj4cld5dkvtqpy9bqc0000gn/T/ipykernel_75146/767564955.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FRONTIER_LOOKUP['SIC'] = FRONTIER_LOOKUP['SIC'].astype(str)


,DATE,INDUSTRY_CODE,EMPLOYMENT,SIC_2digit,SIC_3digit,SIC_4digit,SIC_5digit,IS8_Final_Flag,Frontier_Final_Flag
33,2015,2100,12000,21,210,2100,2100,Life Sciences,NaN
44,2015,6100,14000,61,610,6100,6100,Digital and Technology,NaN
45,2015,6200,1250,62,620,6200,6200,Digital and Technology,NaN
46,2015,7100,0,71,710,7100,7100,Professional and Business Services,NaN
47,2015,7210,0,72,721,7210,7210,Digital and Technology,NaN
...,...,...,...,...,...,...,...,...,...
7444,2024,91011,26000,91,910,9101,91011,Creative Industries,NaN
7445,2024,91012,3500,91,910,9101,91012,Creative Industries,NaN
7446,2024,91020,37000,91,910,9102,91020,Creative Industries,NaN
7463,2024,95110,28000,95,951,9511,95110,Digital and Technology,NaN


In [6]:
# Filter employment DF to only advanced manufacturing frontier industries

advanced_manu_df = df_final[df_final['IS8_Final_Flag']=='Advanced manufacturing']

advanced_manu_df

,DATE,INDUSTRY_CODE,EMPLOYMENT,SIC_2digit,SIC_3digit,SIC_4digit,SIC_5digit,IS8_Final_Flag,Frontier_Final_Flag
146,2015,20110,3500,20,201,2011,20110,Advanced manufacturing,NaN
147,2015,20120,3000,20,201,2012,20120,Advanced manufacturing,NaN
148,2015,20130,4500,20,201,2013,20130,Advanced manufacturing,NaN
149,2015,20140,9000,20,201,2014,20140,Advanced manufacturing,NaN
150,2015,20150,2000,20,201,2015,20150,Advanced manufacturing,NaN
...,...,...,...,...,...,...,...,...,...
7020,2024,30300,75000,30,303,3030,30300,Advanced manufacturing,Aerospace manufacturing
7022,2024,30910,1500,30,309,3091,30910,Advanced manufacturing,NaN
7023,2024,30920,2000,30,309,3092,30920,Advanced manufacturing,NaN
7024,2024,30990,900,30,309,3099,30990,Advanced manufacturing,NaN


In [7]:
# Calculate total advanced manufacturing employment and join back on as a column
advanced_manu_df['Total_Employment_By_Year'] = advanced_manu_df.groupby('DATE')['EMPLOYMENT'].transform('sum')
advanced_manu_df

/var/folders/dp/dmj0l1fj4cld5dkvtqpy9bqc0000gn/T/ipykernel_75146/1575281876.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  advanced_manu_df['Total_Employment_By_Year'] = advanced_manu_df.groupby('DATE')['EMPLOYMENT'].transform('sum')


,DATE,INDUSTRY_CODE,EMPLOYMENT,SIC_2digit,SIC_3digit,SIC_4digit,SIC_5digit,IS8_Final_Flag,Frontier_Final_Flag,Total_Employment_By_Year
146,2015,20110,3500,20,201,2011,20110,Advanced manufacturing,NaN,633200
147,2015,20120,3000,20,201,2012,20120,Advanced manufacturing,NaN,633200
148,2015,20130,4500,20,201,2013,20130,Advanced manufacturing,NaN,633200
149,2015,20140,9000,20,201,2014,20140,Advanced manufacturing,NaN,633200
150,2015,20150,2000,20,201,2015,20150,Advanced manufacturing,NaN,633200
...,...,...,...,...,...,...,...,...,...,...
7020,2024,30300,75000,30,303,3030,30300,Advanced manufacturing,Aerospace manufacturing,587550
7022,2024,30910,1500,30,309,3091,30910,Advanced manufacturing,NaN,587550
7023,2024,30920,2000,30,309,3092,30920,Advanced manufacturing,NaN,587550
7024,2024,30990,900,30,309,3099,30990,Advanced manufacturing,NaN,587550


In [8]:
# Group by frontier sector within advanced manufacturing whilst retaining the total employment for the wider sector

df_grouped = advanced_manu_df.groupby(['DATE','Frontier_Final_Flag']).agg(
    # New column name 'Group_Employment_Sum' will be the sum of 'EMPLOYMENT'
    Group_Employment_Sum=('EMPLOYMENT', 'sum'),
    
    # New column 'Total_Advanced_Manu_Employment' will take the 'first' value it finds
    Total_Advanced_Manu_Employment=('Total_Employment_By_Year', 'first')
).reset_index()

# Calculate frontier share of employment
df_grouped['Share of advanced manufacturing employment'] = df_grouped['Group_Employment_Sum'] / df_grouped['Total_Advanced_Manu_Employment']

df_grouped

,DATE,Frontier_Final_Flag,Group_Employment_Sum,Total_Advanced_Manu_Employment,Share of advanced manufacturing employment
0,2015,Aerospace manufacturing,102000,633200,0.161087
1,2015,Agritech,6800,633200,0.010739
2,2015,Automotive manufacturing,156500,633200,0.247157
3,2015,Batteries,2000,633200,0.003159
4,2016,Aerospace manufacturing,108000,624800,0.172855
5,2016,Agritech,7000,624800,0.011204
6,2016,Automotive manufacturing,162000,624800,0.259283
7,2016,Batteries,1750,624800,0.002801
8,2017,Aerospace manufacturing,100000,633050,0.157965
9,2017,Agritech,6400,633050,0.010110


In [9]:
# Chart the advanced manufacturing sub sectors

chart = alt.Chart(df_grouped).mark_line().encode(
    x=alt.X('DATE:O'),
    y=alt.Y('Share of advanced manufacturing employment:Q', axis=alt.Axis(format='%'),
            title='Share of UK advanced manufacturing employment'),
    color=alt.Color('Frontier_Final_Flag:N', title=None, legend=None)
)


# Add end labels
end_point = df_grouped.groupby(
    ['Frontier_Final_Flag']
)['DATE'].idxmax()

end_point_data = df_grouped.loc[end_point]

# Shift labels up slightly to avoid overlap
end_point_data.loc[end_point_data['Frontier_Final_Flag'] == 'Agritech', 'Share of advanced manufacturing employment'] += 0.005
end_point_data.loc[end_point_data['Frontier_Final_Flag'] == 'Batteries', 'Share of advanced manufacturing employment'] += 0.002


text_labels = alt.Chart(end_point_data).mark_text(
    align='left',     
    dx=5,           
    baseline='middle'
).encode(
    x=alt.X('DATE:O'),         
    y=alt.Y('Share of advanced manufacturing employment:Q', axis=alt.Axis(format='%')),   
    text='Frontier_Final_Flag:N',
    color=alt.Color('Frontier_Final_Flag:N')
)

chart = chart + text_labels
chart
chart.save('advanced_manu.png', scale_factor=2)
chart.save('advanced_manu.json')
